# Finite differences simulation of the JAERI model
Designed to run on a CUDA-compatible GPU.

In [ ]:
import itertools
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep

In [ ]:
plt.style.use(hep.style.ROOT)

## Checking the GPU and compiling the code
Printing the GPU details and compiling the code.

In [ ]:
!nvidia-smi
!nvcc  -o radicals -x cu -lnvToolsExt -I ../CLI11/include ../src/solverRadicals.cu

Checking the executable.

In [ ]:
!./radicals -h

## Running the simulation

Adding a function that runs the simulation and saves the results to a file.

In [2]:
def run_sim(D, R, k1, k2):
    """
    Run the simulation with the given parameters and save the maximum of the activity as function of time as a numpy array.

    Parameters
    ----------
    D : float
        Diffusion coefficient
    R : float
        Radical formation rate (includes the dose rate so it's Y*R)
    k1 : float
        Rate constant for the radical crosslinking reaction
    k2 : float
        Rate constant for the radical oxidation reaction

    Returns
    -------
    None
    """
    print(f"Processing D={D} R={R} k1={k1} k2={k2}")
    process = subprocess.run(
        [
            "./radicals",
            "--diffCoeff",
            str(D),
            "--radFormRate",
            str(R),
            "--k1",
            str(k1),
            "--k2",
            str(k2),
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )
    activity = np.fromfile("outputactivity.dat", dtype=np.float32)
    tDim = 20000
    xDim = 100
    activity = np.reshape(activity, (tDim, xDim))
    maximum = []
    times = range(10001, 20000)
    for time in times:
        maximum.append(np.argmax(activity[time, 250, :]))
    times = np.array(times)
    times = times - times[0]
    maximum = np.array(maximum)
    maximum = maximum / 10
    np.save(f"test_D_{D}_R_{R}_k1_{k1}_k2_{k2}".replace(".", "p"), maximum)

Plot the result of the simulation vs the data.

In [ ]:
def plot_data_vs_sim(D, R, k1, k2):
    """
    Plot the simulation results and the experimental data.

    Parameters
    ----------
    D : float
        Diffusion coefficient
    R : float
        Radical formation rate (includes the dose rate so it's Y*R)
    k1 : float
        Rate constant for the radical crosslinking reaction
    k2 : float
        Rate constant for the radical oxidation reaction

    Returns
    -------
    None
    """
    filename = f"D_{D}_R_{R}_k1_{k1}_k2_{k2}".replace(".", "p")
    label = f"R={R} k1={k1} k2={k2}"
    maximum = np.load(f"test_{filename}.npy")

    times = np.arange(10001, 20000)
    times = times - times[0]

    # find the saturation point
    t_saturation = times[np.argmax(maximum)]

    data_t = np.array([1, 5, 11, 18, 25])
    data_et = np.ones_like(data_t) * 0.5
    data_z = np.array([1.59, 2.09, 2.68, 3.32, 4.09])
    data_ez = np.array([0.02, 0.04, 0.07, 0.11, 0.21])

    z_index = np.array([1.23, 1.23, 1.24, 1.16, 1.14])
    z_index_mean = np.mean(z_index)
    z_index_std = np.sqrt((np.std(z_index)) ** 2 + (0.05 * z_index_mean) ** 2)

    # Days to a.u.
    data_t = data_t * (t_saturation * 0.85 / data_t[-1])

    # Calculate chi^2
    chi2 = np.sum((data_z - maximum[(np.rint(data_t)).astype(int)]) ** 2 / data_ez)
    chi2 += (z_index_mean - maximum[0]) ** 2 / z_index_std

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.plot(times, maximum, label=label)
    ax.errorbar(data_t, data_z, xerr=data_et, yerr=data_ez)
    ax.hlines(
        z_index_mean,
        -1000,
        10000,
        color="black",
        linestyles="dashed",
        label="index boundary",
        linewidth=2,
    )
    ax.fill_between(
        np.linspace(-1000, 10000, 100),
        z_index_mean - z_index_std,
        z_index_mean + z_index_std,
        alpha=0.2,
        color="black",
    )
    ax.text(t_saturation * 0.7, 0.3, f"$\chi^2 = ${chi2:.4f}")
    ax.set_xlim(-100, t_saturation * 1.1)
    ax.set_ylim(0, 5)
    ax.set_xlabel("t (a.u.)")
    ax.set_ylabel("z (mm)")
    fig.tight_layout()
    plt.legend(loc="upper left")
    plt.show()

Now run the code.

In [ ]:
# These parameters will make the simulation match the experimental data
D = 0.1
R = 0.0006
k1 = 0.02
k2 = 0.2

run_sim(D, R, k1, k2)
plot_data_vs_sim(D, R, k1, k2)